In [1]:
import random
import sys
import numpy as np
import pandas as pd

from sklearn.preprocessing import (StandardScaler, MinMaxScaler)

from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor

from sklearn.metrics import (mean_absolute_error, mean_squared_error)


In [2]:
df = pd.read_excel("NFL Champion RawData.xlsx")
df.head(2)

,Season,Team ID,Team ID/Season,Team,Team/Season,sum mvp share,sum dpoy share,sum opoy share,sum all pro,sum cpoy share,...,team_defense_Rushing_Yds,team_defense_Rushing_TD,team_defense_Rushing_Y/A,team_defense_Rushing_1stD,team_defense_Penalties_Pen,team_defense_Penalties_Yds,team_defense_Penalties_1stPy,team_defense_Sc%,team_defense_TO%,team_defense_EXP
0,2023,1,1 2022,Kansas City Chiefs,Kansas City Chiefs 2022,0.86,0.0,0.78,9,0.0,...,1823,10,4.4,102,98,734,28,35.9,10.3,-65.59
1,2023,17,17 2022,Philadelphia Eagles,Philadelphia Eagles 2022,0.00,0.0,0.00,22,0.0,...,2068,15,4.6,115,95,798,24,32.4,15.1,20.73


In [3]:
co = df
print(list(co.columns))

['Season', 'Team ID', ' Team ID/Season', 'Team', 'Team/Season', 'sum mvp share', 'sum dpoy share', 'sum opoy share', 'sum all pro', 'sum cpoy share', 'sum playoff games', 'sum champion', 'L1Y cs', 'L3Y cs', 'L5Y cs', 'L8Y cs', 'W', 'L', 'T', 'W-L%', 'PF', 'PA', 'PD', 'MoV', 'SoS', 'SRS', 'OSRS', 'DSRS', 'make playoffs', 'rk playoffs', '1 seed', '2 seed', '3 seed', '4 seed', '5 seed', '6 seed', 'champion share', 'pso', 'team_offense_G', 'team_offense_PF', 'team_offense_Tot Yds & TO_Yds', 'team_offense_Tot Yds & TO_Ply', 'team_offense_Tot Yds & TO_Y/P', 'team_offense_Tot Yds & TO_TO', 'team_offense_Tot Yds & TO_FL', 'team_offense_Tot Yds & TO_1stD', 'team_offense_Tot Yds & TO_Cmp', 'team_offense_Tot Yds & TO_Att', 'team_offense_Passing_Yds', 'team_offense_Passing_TD', 'team_offense_Passing_Int', 'team_offense_Passing_NY/A', 'team_offense_Passing_1stD', 'team_offense_Passing_Att', 'team_offense_Rushing_Yds', 'team_offense_Rushing_TD', 'team_offense_Rushing_Y/A', 'team_offense_Rushing_1stD

In [4]:
#Dropping all data before 2001, as this dataset begins in 1991, so stats don't fully come into focus until 2001s.
#All data cleaning was done prior in excel

df.drop(df[df['Season'] < 2001].index, inplace = True)
df

,Season,Team ID,Team ID/Season,Team,Team/Season,sum mvp share,sum dpoy share,sum opoy share,sum all pro,sum cpoy share,...,team_defense_Rushing_Yds,team_defense_Rushing_TD,team_defense_Rushing_Y/A,team_defense_Rushing_1stD,team_defense_Penalties_Pen,team_defense_Penalties_Yds,team_defense_Penalties_1stPy,team_defense_Sc%,team_defense_TO%,team_defense_EXP
0,2023,1,1 2022,Kansas City Chiefs,Kansas City Chiefs 2022,0.860,0.000,0.780,9,0.00,...,1823,10,4.4,102,98,734,28,35.9,10.3,-65.59
1,2023,17,17 2022,Philadelphia Eagles,Philadelphia Eagles 2022,0.000,0.000,0.000,22,0.00,...,2068,15,4.6,115,95,798,24,32.4,15.1,20.73
2,2023,19,19 2022,San Francisco 49ers,San Francisco 49ers 2022,0.000,0.000,0.240,10,0.30,...,1321,11,3.4,77,96,832,26,25.7,15.3,56.26
3,2023,2,2 2022,Buffalo Bills,Buffalo Bills 2022,0.080,0.380,0.060,16,0.16,...,1673,10,4.3,103,89,754,29,31.0,14.3,4.46
4,2023,18,18 2022,Minnesota Vikings,Minnesota Vikings 2022,0.000,0.020,0.000,21,0.02,...,2093,18,4.5,113,111,926,22,41.1,12.0,-104.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
729,2001,3,3 2000,Cincinnati Bengals,Cincinnati Bengals 2000,0.000,0.000,0.000,2,0.00,...,1925,12,3.8,101,115,924,24,32.8,9.7,136.24
730,2001,29,29 2000,Atlanta Falcons,Atlanta Falcons 2000,0.085,0.000,0.043,7,0.00,...,1983,16,4.4,113,122,1010,20,37.0,12.5,88.81
731,2001,11,11 2000,Cleveland Browns,Cleveland Browns 2000,0.000,0.000,0.000,0,0.00,...,2505,26,4.2,147,71,626,27,39.1,10.9,49.34
732,2001,31,31 2000,Arizona Cardinals,Arizona Cardinals 2000,0.000,0.000,0.000,6,0.00,...,2609,29,4.5,149,96,914,21,42.1,11.2,25.06


Model Workings:
    
1st Approach: Past Testing, predicting the last 10 nfl champions as accurately as possible. (2023-2014)

2nd Approach: Current Testing, training all data to year in which prediction is needed.

In [5]:
co = df
print(list(co.columns))

['Season', 'Team ID', ' Team ID/Season', 'Team', 'Team/Season', 'sum mvp share', 'sum dpoy share', 'sum opoy share', 'sum all pro', 'sum cpoy share', 'sum playoff games', 'sum champion', 'L1Y cs', 'L3Y cs', 'L5Y cs', 'L8Y cs', 'W', 'L', 'T', 'W-L%', 'PF', 'PA', 'PD', 'MoV', 'SoS', 'SRS', 'OSRS', 'DSRS', 'make playoffs', 'rk playoffs', '1 seed', '2 seed', '3 seed', '4 seed', '5 seed', '6 seed', 'champion share', 'pso', 'team_offense_G', 'team_offense_PF', 'team_offense_Tot Yds & TO_Yds', 'team_offense_Tot Yds & TO_Ply', 'team_offense_Tot Yds & TO_Y/P', 'team_offense_Tot Yds & TO_TO', 'team_offense_Tot Yds & TO_FL', 'team_offense_Tot Yds & TO_1stD', 'team_offense_Tot Yds & TO_Cmp', 'team_offense_Tot Yds & TO_Att', 'team_offense_Passing_Yds', 'team_offense_Passing_TD', 'team_offense_Passing_Int', 'team_offense_Passing_NY/A', 'team_offense_Passing_1stD', 'team_offense_Passing_Att', 'team_offense_Rushing_Yds', 'team_offense_Rushing_TD', 'team_offense_Rushing_Y/A', 'team_offense_Rushing_1stD

Features removed from model: (reasoning: different features change model performance).

Identificaiton columns: 'Season', 'Team ID', ' Team ID/Season', 'Team', 'Team/Season',
    
Prediction column:  'champion share',   
    
Data columns: 'W', 'L', 'T', 'W-L%' , 'make playoffs',
            'rk playoffs', 'PD', 'MoV'

General Rule of Machine Learning Thumb: 5 Features = 50 Rows, we have 734 rows in our entire set used = we need 73 features

In [6]:
#RandomForest Regressor
#PastApproach

features = ['SRS','1 seed', '2 seed', '3 seed', '4 seed', '5 seed', '6 seed','sum mvp share', 'sum dpoy share', 
            'sum opoy share', 'sum all pro', 'sum cpoy share', 'sum playoff games', 'sum champion', 'L1Y cs', 'L3Y cs',
            'L5Y cs', 'L8Y cs','PF', 'PA', 'SoS', 'OSRS', 'DSRS',   'pso', 'team_offense_G',
            'team_offense_PF', 'team_offense_Tot Yds & TO_Yds', 'team_offense_Tot Yds & TO_Ply', 'team_offense_Tot Yds & TO_Y/P', 
            'team_offense_Tot Yds & TO_TO', 'team_offense_Tot Yds & TO_FL', 'team_offense_Tot Yds & TO_1stD', 'team_offense_Tot Yds & TO_Cmp',
            'team_offense_Tot Yds & TO_Att', 'team_offense_Passing_Yds', 'team_offense_Passing_TD', 'team_offense_Passing_Int', 'team_offense_Passing_NY/A', 
            'team_offense_Passing_1stD', 'team_offense_Passing_Att', 'team_offense_Rushing_Yds', 'team_offense_Rushing_TD', 'team_offense_Rushing_Y/A', 
            'team_offense_Rushing_1stD', 'team_offense_Penalties_Pen', 'team_offense_Penalties_Yds', 'team_offense_Penalties_1stPy', 'team_offense_Sc%', 
            'team_offense_TO%', 'team_offense_EXP', 'team_defense_G', 'team_defense_PA', 'team_defense_Tot Yds & TO_Yds', 'team_defense_Tot Yds & TO_Ply', 
            'team_defense_Tot Yds & TO_Y/P', 'team_defense_Tot Yds & TO_TO', 'team_defense_Tot Yds & TO_FL', 'team_defense_Tot Yds & TO_1stD', 
            'team_defense_Tot Yds & TO_Cmp', 'team_defense_Tot Yds & TO_Att', 'team_defense_Passing_Yds', 'team_defense_Passing_TD', 
            'team_defense_Passing_Int', 'team_defense_Passing_NY/A', 'team_defense_Passing_1stD', 'team_defense_Passing_Att', 'team_defense_Rushing_Yds', 
            'team_defense_Rushing_TD', 'team_defense_Rushing_Y/A', 'team_defense_Rushing_1stD', 'team_defense_Penalties_Pen', 'team_defense_Penalties_Yds', 
            'team_defense_Penalties_1stPy', 'team_defense_Sc%', 'team_defense_TO%', 'team_defense_EXP']

#training/testing
train = df[df["Season"] < 2013].copy()
test = df[df["Season"] > 2013].copy()

#scaling
sc = MinMaxScaler()
sc.fit(train[features])
train[features]=sc.transform(train[features])
test[features]=sc.transform(test[features])

#model parameters
rf = RandomForestRegressor(n_estimators=500, random_state=50, min_samples_split=2, max_depth=5)
rf.fit(train[features], train["champion share"])

#predicting
predictions = rf.predict(test[features]) 
predictions = pd.DataFrame(predictions, columns=["predicted champion share"], index=test.index)
frame = pd.concat([test[["Season", "Team", "champion share"]], predictions], axis=1)
frame.sort_values(["Season", "predicted champion share"], inplace=True, ascending = [False, False])
#sc.transform(df[features])

#result preview
print("Results:")
print("=============================================================================\n")
print(frame.head(5))
print("-----------------------------------------------------------------------------\n")

#error metrics
mae = mean_absolute_error(frame["champion share"], frame["predicted champion share"])
mse = mean_squared_error(frame["champion share"], frame["predicted champion share"])
rmse = mean_squared_error(frame["champion share"], frame["predicted champion share"], squared=False)
print("Error:")
print("=============================================================================\n")
print("MAE: ", mae)
print("MSE: ", mse)
print("RMSE: ", rmse)
print("-----------------------------------------------------------------------------\n")

#feature importance
f_list = list(features)
f_importance = pd.Series(rf.feature_importances_, index = f_list).sort_values(ascending = False)
print("Feature Importance: ")
print("=============================================================================\n")
print(f_importance.head(10))
print("-----------------------------------------------------------------------------\n")

#results in full
print("Full Results:")
print("=============================================================================\n")
predictions = rf.predict(test[features]) 
predictions = pd.DataFrame(predictions*100, columns=["predicted champion share"], index=test.index)
frame = pd.concat([test[["Season", "Team", "champion share"]], predictions], axis=1)
frame.sort_values(["Season", "predicted champion share"], inplace=True, ascending = [False, False])

print(frame.to_string())

Results:

   Season                 Team  champion share  predicted champion share
0    2023   Kansas City Chiefs            1.00                  0.512296
3    2023        Buffalo Bills            0.25                  0.413830
1    2023  Philadelphia Eagles            0.75                  0.397455
2    2023  San Francisco 49ers            0.50                  0.386103
5    2023   Cincinnati Bengals            0.50                  0.248893
-----------------------------------------------------------------------------

Error:

MAE:  0.09606652201114668
MSE:  0.026182498975507118
RMSE:  0.1618100706863053
-----------------------------------------------------------------------------

Feature Importance: 

SRS                         0.336020
2 seed                      0.083910
1 seed                      0.060295
PA                          0.044102
team_defense_PA             0.037668
team_offense_Passing_Yds    0.031017
team_defense_EXP            0.030649
DSRS                      

In [18]:
#Ridge Regressor
#PastApproach

features = ['SRS','1 seed', '2 seed', '3 seed', '4 seed', '5 seed', '6 seed','sum mvp share', 'sum dpoy share', 
            'sum opoy share', 'sum all pro', 'sum cpoy share', 'sum playoff games', 'sum champion', 'L1Y cs', 'L3Y cs',
            'L5Y cs', 'L8Y cs','PF', 'PA', 'SoS', 'OSRS', 'DSRS',   'pso', 'team_offense_G',
            'team_offense_PF', 'team_offense_Tot Yds & TO_Yds', 'team_offense_Tot Yds & TO_Ply', 'team_offense_Tot Yds & TO_Y/P', 
            'team_offense_Tot Yds & TO_TO', 'team_offense_Tot Yds & TO_FL', 'team_offense_Tot Yds & TO_1stD', 'team_offense_Tot Yds & TO_Cmp',
            'team_offense_Tot Yds & TO_Att', 'team_offense_Passing_Yds', 'team_offense_Passing_TD', 'team_offense_Passing_Int', 'team_offense_Passing_NY/A', 
            'team_offense_Passing_1stD', 'team_offense_Passing_Att', 'team_offense_Rushing_Yds', 'team_offense_Rushing_TD', 'team_offense_Rushing_Y/A', 
            'team_offense_Rushing_1stD', 'team_offense_Penalties_Pen', 'team_offense_Penalties_Yds', 'team_offense_Penalties_1stPy', 'team_offense_Sc%', 
            'team_offense_TO%', 'team_offense_EXP', 'team_defense_G', 'team_defense_PA', 'team_defense_Tot Yds & TO_Yds', 'team_defense_Tot Yds & TO_Ply', 
            'team_defense_Tot Yds & TO_Y/P', 'team_defense_Tot Yds & TO_TO', 'team_defense_Tot Yds & TO_FL', 'team_defense_Tot Yds & TO_1stD', 
            'team_defense_Tot Yds & TO_Cmp', 'team_defense_Tot Yds & TO_Att', 'team_defense_Passing_Yds', 'team_defense_Passing_TD', 
            'team_defense_Passing_Int', 'team_defense_Passing_NY/A', 'team_defense_Passing_1stD', 'team_defense_Passing_Att', 'team_defense_Rushing_Yds', 
            'team_defense_Rushing_TD', 'team_defense_Rushing_Y/A', 'team_defense_Rushing_1stD', 'team_defense_Penalties_Pen', 'team_defense_Penalties_Yds', 
            'team_defense_Penalties_1stPy', 'team_defense_Sc%', 'team_defense_TO%', 'team_defense_EXP']

#training/testing
train = df[df["Season"] < 2013].copy()
test = df[df["Season"] > 2013].copy()

#ssc = MinMaxScaler()caling
sc.fit(train[features])
train[features]=sc.transform(train[features])
test[features]=sc.transform(test[features])

#model parameters
reg = Ridge(alpha=.1)
reg.fit(train[features], train['champion share'])

#predicting
predictions = reg.predict(test[features]) 
predictions = pd.DataFrame(predictions, columns=["predicted champion share"], index=test.index)
frame = pd.concat([test[["Season", "Team", "champion share"]], predictions], axis=1)
frame.sort_values(["Season", "predicted champion share"], inplace=True, ascending = [False, False])

#result preview
print("Results:")
print("=============================================================================\n")
print(frame.head(5))
print("-----------------------------------------------------------------------------\n")

#error metrics
mae = mean_absolute_error(frame["champion share"], frame["predicted champion share"])
mse = mean_squared_error(frame["champion share"], frame["predicted champion share"])
rmse = mean_squared_error(frame["champion share"], frame["predicted champion share"], squared=False)
print("Error:")
print("=============================================================================\n")
print("MAE: ", mae)
print("MSE: ", mse)
print("RMSE: ", rmse)
print("-----------------------------------------------------------------------------\n")

#feature importance
f_list = list(features)
f_importance = pd.Series(reg.coef_, index = f_list).sort_values(ascending = False)
print("Feature Importance: ")
print("=============================================================================\n")
print(f_importance.head(10))
print("-----------------------------------------------------------------------------\n")

#results in full
print("Full Results:")
print("=============================================================================\n")
predictions = reg.predict(test[features]) 
predictions = pd.DataFrame(predictions*100, columns=["predicted champion share"], index=test.index)
frame = pd.concat([test[["Season", "Team", "champion share"]], predictions], axis=1)
frame.sort_values(["Season", "predicted champion share"], inplace=True, ascending = [False, False])

#print(frame.to_string())

Results:

   Season                  Team  champion share  predicted champion share
3    2023         Buffalo Bills            0.25                  0.548562
1    2023   Philadelphia Eagles            0.75                  0.540193
2    2023   San Francisco 49ers            0.50                  0.455022
0    2023    Kansas City Chiefs            1.00                  0.411059
9    2023  Jacksonville Jaguars            0.25                  0.167330
-----------------------------------------------------------------------------

Error:

MAE:  0.1363676363168397
MSE:  0.03426341551677131
RMSE:  0.18510379660280152
-----------------------------------------------------------------------------

Feature Importance: 

team_defense_EXP                 0.552821
1 seed                           0.513644
sum playoff games                0.507917
2 seed                           0.495016
team_offense_TO%                 0.392049
team_defense_Rushing_Yds         0.375131
team_defense_Tot Yds & TO_Yd

In [19]:
#XG Boost Regressor
#PastApproach

features = ['SRS','1 seed', '2 seed', '3 seed', '4 seed', '5 seed', '6 seed','sum mvp share', 'sum dpoy share', 
            'sum opoy share', 'sum all pro', 'sum cpoy share', 'sum playoff games', 'sum champion', 'L1Y cs', 'L3Y cs',
            'L5Y cs', 'L8Y cs','PF', 'PA', 'SoS', 'OSRS', 'DSRS',   'pso', 'team_offense_G',
            'team_offense_PF', 'team_offense_Tot Yds & TO_Yds', 'team_offense_Tot Yds & TO_Ply', 'team_offense_Tot Yds & TO_Y/P', 
            'team_offense_Tot Yds & TO_TO', 'team_offense_Tot Yds & TO_FL', 'team_offense_Tot Yds & TO_1stD', 'team_offense_Tot Yds & TO_Cmp',
            'team_offense_Tot Yds & TO_Att', 'team_offense_Passing_Yds', 'team_offense_Passing_TD', 'team_offense_Passing_Int', 'team_offense_Passing_NY/A', 
            'team_offense_Passing_1stD', 'team_offense_Passing_Att', 'team_offense_Rushing_Yds', 'team_offense_Rushing_TD', 'team_offense_Rushing_Y/A', 
            'team_offense_Rushing_1stD', 'team_offense_Penalties_Pen', 'team_offense_Penalties_Yds', 'team_offense_Penalties_1stPy', 'team_offense_Sc%', 
            'team_offense_TO%', 'team_offense_EXP', 'team_defense_G', 'team_defense_PA', 'team_defense_Tot Yds & TO_Yds', 'team_defense_Tot Yds & TO_Ply', 
            'team_defense_Tot Yds & TO_Y/P', 'team_defense_Tot Yds & TO_TO', 'team_defense_Tot Yds & TO_FL', 'team_defense_Tot Yds & TO_1stD', 
            'team_defense_Tot Yds & TO_Cmp', 'team_defense_Tot Yds & TO_Att', 'team_defense_Passing_Yds', 'team_defense_Passing_TD', 
            'team_defense_Passing_Int', 'team_defense_Passing_NY/A', 'team_defense_Passing_1stD', 'team_defense_Passing_Att', 'team_defense_Rushing_Yds', 
            'team_defense_Rushing_TD', 'team_defense_Rushing_Y/A', 'team_defense_Rushing_1stD', 'team_defense_Penalties_Pen', 'team_defense_Penalties_Yds', 
            'team_defense_Penalties_1stPy', 'team_defense_Sc%', 'team_defense_TO%', 'team_defense_EXP']

#training/testing
train = df[df["Season"] < 2013].copy()
test = df[df["Season"] > 2013].copy()

#scaling
sc = MinMaxScaler()
sc.fit(train[features])
train[features]=sc.transform(train[features])
test[features]=sc.transform(test[features])

#model parameters
xgb = XGBRegressor(n_estimators=500,learning_rate=.05)
xgb.fit(train[features], train['champion share'])

#predicting
predictions = xgb.predict(test[features]) 
predictions = pd.DataFrame(predictions, columns=["predicted champion share"], index=test.index)
frame = pd.concat([test[["Season", "Team", "champion share"]], predictions], axis=1)
frame.sort_values(["Season", "predicted champion share"], inplace=True, ascending = [False, False])

#result preview
print("Results:")
print("=============================================================================\n")
print(frame.head(5))
print("-----------------------------------------------------------------------------\n")

#error metrics
mae = mean_absolute_error(frame["champion share"], frame["predicted champion share"])
mse = mean_squared_error(frame["champion share"], frame["predicted champion share"])
rmse = mean_squared_error(frame["champion share"], frame["predicted champion share"], squared=False)
print("Error:")
print("=============================================================================\n")
print("MAE: ", mae)
print("MSE: ", mse)
print("RMSE: ", rmse)
print("-----------------------------------------------------------------------------\n")

#feature importance
f_list = list(features)
f_importance = pd.Series(xgb.feature_importances_, index = f_list).sort_values(ascending = False)
print("Feature Importance: ")
print("=============================================================================\n")
print(f_importance.head(10))
print("-----------------------------------------------------------------------------\n")

#results in full
print("Full Results:")
print("=============================================================================\n")
predictions = xgb.predict(test[features]) 
predictions = pd.DataFrame(predictions*100, columns=["predicted champion share"], index=test.index)
frame = pd.concat([test[["Season", "Team", "champion share"]], predictions], axis=1)
frame.sort_values(["Season", "predicted champion share"], inplace=True, ascending = [False, False])

#print(frame.to_string())

Results:

    Season                  Team  champion share  predicted champion share
0     2023    Kansas City Chiefs            1.00                  0.556037
18    2023  Tampa Bay Buccaneers            0.00                  0.457922
1     2023   Philadelphia Eagles            0.75                  0.433863
2     2023   San Francisco 49ers            0.50                  0.412527
9     2023  Jacksonville Jaguars            0.25                  0.392775
-----------------------------------------------------------------------------

Error:

MAE:  0.09775478211436166
MSE:  0.029733483879732004
RMSE:  0.17243399861898467
-----------------------------------------------------------------------------

Feature Importance: 

1 seed                           0.125051
PA                               0.096927
2 seed                           0.090434
team_offense_Tot Yds & TO_Cmp    0.058052
SRS                              0.055016
DSRS                             0.048253
team_offense_Tot Yds

Present Test: 

In [20]:
#RandomForest Regressor
#PresentTest Approach

features = ['1 seed', '2 seed', '3 seed', '4 seed', '5 seed', '6 seed','sum mvp share', 'sum dpoy share', 
            'sum opoy share', 'sum all pro', 'sum cpoy share', 'sum playoff games', 'sum champion', 'L1Y cs', 'L3Y cs',
            'L5Y cs', 'L8Y cs','PF', 'PA','SoS', 'SRS', 'OSRS', 'DSRS',   'pso', 'team_offense_G',
            'team_offense_PF', 'team_offense_Tot Yds & TO_Yds', 'team_offense_Tot Yds & TO_Ply', 'team_offense_Tot Yds & TO_Y/P', 
            'team_offense_Tot Yds & TO_TO', 'team_offense_Tot Yds & TO_FL', 'team_offense_Tot Yds & TO_1stD', 'team_offense_Tot Yds & TO_Cmp',
            'team_offense_Tot Yds & TO_Att', 'team_offense_Passing_Yds', 'team_offense_Passing_TD', 'team_offense_Passing_Int', 'team_offense_Passing_NY/A', 
            'team_offense_Passing_1stD', 'team_offense_Passing_Att', 'team_offense_Rushing_Yds', 'team_offense_Rushing_TD', 'team_offense_Rushing_Y/A', 
            'team_offense_Rushing_1stD', 'team_offense_Penalties_Pen', 'team_offense_Penalties_Yds', 'team_offense_Penalties_1stPy', 'team_offense_Sc%', 
            'team_offense_TO%', 'team_offense_EXP', 'team_defense_G', 'team_defense_PA', 'team_defense_Tot Yds & TO_Yds', 'team_defense_Tot Yds & TO_Ply', 
            'team_defense_Tot Yds & TO_Y/P', 'team_defense_Tot Yds & TO_TO', 'team_defense_Tot Yds & TO_FL', 'team_defense_Tot Yds & TO_1stD', 
            'team_defense_Tot Yds & TO_Cmp', 'team_defense_Tot Yds & TO_Att', 'team_defense_Passing_Yds', 'team_defense_Passing_TD', 
            'team_defense_Passing_Int', 'team_defense_Passing_NY/A', 'team_defense_Passing_1stD', 'team_defense_Passing_Att', 'team_defense_Rushing_Yds', 
            'team_defense_Rushing_TD', 'team_defense_Rushing_Y/A', 'team_defense_Rushing_1stD', 'team_defense_Penalties_Pen', 'team_defense_Penalties_Yds', 
            'team_defense_Penalties_1stPy', 'team_defense_Sc%', 'team_defense_TO%', 'team_defense_EXP']

#training/testing
train = df[df["Season"] < 2022].copy()
test = df[df["Season"] > 2022].copy()

#scaling
sc = MinMaxScaler()
sc.fit(train[features])
train[features]=sc.transform(train[features])
test[features]=sc.transform(test[features])

#model parameters
rf = RandomForestRegressor(n_estimators=500, random_state=50, min_samples_split=2, max_depth=5)
rf.fit(train[features], train["champion share"])

#predicting
predictions = rf.predict(test[features]) 
predictions = pd.DataFrame(predictions, columns=["predicted champion share"], index=test.index)
frame = pd.concat([test[["Season", "Team", "champion share"]], predictions], axis=1)
frame.sort_values(["Season", "predicted champion share"], inplace=True, ascending = [False, False])

#result preview
print("Result Preview:")
print("=============================================================================\n")
print(frame.head(5))
print("-----------------------------------------------------------------------------\n")

#error metrics

#mae
#error metrics
mae1 = mean_absolute_error(frame["champion share"], frame["predicted champion share"])
mse1 = mean_squared_error(frame["champion share"], frame["predicted champion share"])
rmse1 = mean_squared_error(frame["champion share"], frame["predicted champion share"], squared=False)

#Average Precision Metric by DataQuest
#------------------------------------------------------------------------------------------
#error metric by ranking that I found from dataquest. Useful for ranking problems. 
#Specifically the top 5 or what ranking range chosen, if that team is predicted in top 5 and actually finished top 5 it's 1.0, 100% success, but
#for instance if it took 6 ranks to find the 4th actual ranked entity then that would result in a value of 4/6 so .6667.
#So predicted 33th, but finished top 5 = 5/33 = .1515.

frame = pd.concat([test[["Team", "champion share"]], predictions], axis=1)
frame.sort_values(["champion share"], ascending=False).head(10)
actual = frame.sort_values("champion share", ascending=False)
predicted = frame.sort_values("predicted champion share", ascending=False)
actual["Rank"] = list(range(1,actual.shape[0]+1))
predicted["Predicted Rank"] = list(range(1,predicted.shape[0]+1))
actual.merge(predicted, on="Team").head(5)

def find_ap(frame):
    actual = frame.sort_values("champion share", ascending=False).head(5)
    predicted = frame.sort_values("predicted champion share", ascending=False)
    ps = []
    found = 0
    seen = 1
    for index,row in predicted.iterrows():
        if row["Team"] in actual["Team"].values:
            found += 1
            ps.append(found / seen)
        seen += 1
    return sum(ps) / len(ps)
ap1 = find_ap(frame)

print("Error Metrics: ")
print("=============================================================================\n")
print("PresentTest AP: ", ap1)
print("PresentTest MAE: ", mae1)
print("PresentTest MSE: ", mse1)
print("PresentTest RMSE: ", rmse1)
print("-----------------------------------------------------------------------------\n")
#------------------------------------------------------------------------------------------

#feature importance
f_list = list(features)
f_importance = pd.Series(rf.feature_importances_, index = f_list).sort_values(ascending = False)
print("Feature Importance: ")
print("=============================================================================\n")
print(f_importance.head(10))
print("-----------------------------------------------------------------------------\n")

#results in full
print("Full Results:")
print("=============================================================================\n")
predictions = rf.predict(test[features]) 
predictions = pd.DataFrame(predictions*100, columns=["predicted champion share"], index=test.index)
frame = pd.concat([test[["Season", "Team", "champion share"]], predictions], axis=1)
frame.sort_values(["Season", "predicted champion share"], inplace=True, ascending = [False, False])

print(frame.to_string())

Result Preview:

    Season                  Team  champion share  predicted champion share
0     2023    Kansas City Chiefs            1.00                  0.631677
1     2023   Philadelphia Eagles            0.75                  0.530585
2     2023   San Francisco 49ers            0.50                  0.482136
3     2023         Buffalo Bills            0.25                  0.467898
18    2023  Tampa Bay Buccaneers            0.00                  0.394524
-----------------------------------------------------------------------------

Error Metrics: 

PresentTest AP:  0.8761904761904761
PresentTest MAE:  0.07355579280087099
PresentTest MSE:  0.017256730291460354
PresentTest RMSE:  0.13136487464866836
-----------------------------------------------------------------------------

Feature Importance: 

SRS                         0.453076
1 seed                      0.092828
2 seed                      0.090470
4 seed                      0.022109
DSRS                        0.018360

In [21]:
#Export Prediction 
predictions = rf.predict(test[features]) 
predictions = pd.DataFrame(predictions, columns=["predicted champion share"], index=test.index)

frame = pd.concat([test[['Season', "Team/Season"]], predictions], axis=1)
frame.sort_values(["Season", "predicted champion share"], inplace=True, ascending = [False, False])
frame.to_excel('NFL Champion Predict.xlsx')

#Export Importance
f_list = list(features)
f_importance = pd.Series(rf.feature_importances_, index = f_list).sort_values(ascending = False)
f_importance.to_excel('NFL Champion Fea Imp.xlsx')

In [22]:
#RandomForest Regressor
#PresentApproach
#BackTesting Results


#backtesting
#-------------------------------------------------------------------------------------------------------
#5 = 2006
#all data from 2001-2005 becomes training, and so on.
Seasons = list(range(2001,2024))

aps=[]
all_predictions=[]
for Season in Seasons[5:]:
    train=df[df["Season"]<Season].copy()
    test=df[df["Season"]==Season].copy()

    sc = MinMaxScaler()
    sc.fit(train[features])
    train[features]=sc.transform(train[features])
    test[features]=sc.transform(test[features])
    
    
    rf.fit(train[features], train['champion share'])
            
predictions = rf.predict(test[features]) 
predictions = pd.DataFrame(predictions, columns=["predicted champion share"], index=test.index)
frame = pd.concat([test[["Season", "Team", "champion share"]], predictions], axis=1)
frame.sort_values(["Season", "predicted champion share"], inplace=True, ascending = [False, False])

all_predictions.append(frame)
aps.append(find_ap(frame))

ap2 = sum(aps)/len(aps)

mae2 = mean_absolute_error(frame["champion share"], frame["predicted champion share"])
mse2 = mean_squared_error(frame["champion share"], frame["predicted champion share"])
rmse2 = mean_squared_error(frame["champion share"], frame["predicted champion share"], squared=False)

#-------------------------------------------------------------------------------------------------------

#Error Metrics
print("Error Metrics: ")
print("=============================================================================\n")
print("BackTest AP: ", ap2)
print("BackTest MAE: ", mae2)
print("BackTest MSE: ", mse2)
print("BackTest RMSE: ", rmse2)
print("-----------------------------------------------------------------------------\n")

Error Metrics: 

BackTest AP:  0.925
BackTest MAE:  0.07348597523882176
BackTest MSE:  0.0186348359483512
BackTest RMSE:  0.13650947200964186
-----------------------------------------------------------------------------

